## 14. 如何选择使用哪个模型？

### 场景 1：需要快速判断代码是否有 bug
**推荐**：✅ GEPA 原生模型 (BugPresenceDetector)
- 80% 准确率
- 简单明确的输出（有/无 bug）
- 适合 CI/CD 集成、代码审查工具

### 场景 2：需要知道 bug 的具体类型
**推荐**：✅ 传统模型 (CodeBugDetector) + BootstrapFewShot 优化
- 能分类 10 种 bug 类型
- BootstrapFewShot 优化后准确率 ~70%
- 适合详细分析、学习用途

### 场景 3：需要最高准确率的类别分类
**推荐**：🔄 待实现 - MIPROv2 优化
- 指令 + Few-shot 混合优化
- 预期准确率 50-70%
- 运行：`python python_src/train_mipro_v2.py`（需创建）

### 场景 4：生产环境大规模使用
**推荐**：🔄 混合方案
```python
# 第一步：快速筛选
gepa_result = gepa_detector(code)
if not gepa_result.has_bug:
    return "安全"

# 第二步：详细分类（只对检测到 bug 的代码）
detailed_result = traditional_detector(code)
return detailed_result
```

---

## 🎯 训练新模型的命令

```bash
# 传统模型 + BootstrapFewShot
python python_src/train_optimizer.py

# GEPA 原生模型（已完成）
python python_src/train_gepa_native.py

# MIPROv2（推荐用于类别分类，待创建）
python python_src/train_mipro_v2.py
```

## 13. 传统模型 vs GEPA 原生模型对比

**关键差异**：

| 特性 | 传统模型 (CodeBugDetector) | GEPA 原生模型 (BugPresenceDetector) |
|------|---------------------------|-------------------------------------|
| **任务** | 判断有无 bug + 分类为 10 种类型 | 只判断有无 bug |
| **输出** | has_bug + bug_details (JSON) | has_bug + confidence + reasoning |
| **类型安全** | ❌ 依赖 JSON 解析 | ✅ Pydantic 强制类型 |
| **GEPA 兼容** | ❌ 评估全失败 (0.0 分) | ✅ 完全兼容 |
| **准确率** | 66.7% (未优化) | 80% (GEPA 优化) |
| **优化效果** | BootstrapFewShot: +40% | GEPA: +20% |
| **训练时间** | 5-10 分钟 | 30 分钟 |

**为什么 GEPA 原生模型不做分类？**

1. **任务难度**：即使简化为 3 大类，基础模型也只有 3.3% 准确率
2. **GEPA 局限**：GEPA 只优化指令，不提供示例。而类别分类需要具体示例来学习模式
3. **实用性**：80% 的 bug 检测已经很有价值，可以作为第一道筛选

**如果需要类别分类，建议**：
- 使用 MIPROv2（指令 + Few-shot 混合优化）
- 或者使用更强的基础模型（Gemini 1.5 Pro）+ GEPA
- 或者混合方案：GEPA 检测 + 传统方法分类

In [5]:
# 使用 GEPA 原生模型测试所有示例
if gepa_detector is not None:
    print("=" * 80)
    print("GEPA 原生模型检测结果")
    print("=" * 80)
    print()
    
    test_cases = {
        "示例1: 内存泄漏": code_example_1,
        "示例2: 缓冲区溢出": code_example_2,
        "示例3: 空指针解引用": code_example_3,
        "示例4: 数组越界": code_example_4,
        "示例5: 安全代码（防御性编程）": code_example_5
    }
    
    results = []
    
    for name, code in test_cases.items():
        print(f"{name}")
        print("-" * 80)
        
        try:
            # 使用 GEPA 原生模型
            result = gepa_detector(code_snippet=code)
            
            # 格式化输出
            if result.has_bug:
                print(f"  判断: ✗ 有 bug")
                print(f"  置信度: {result.confidence:.2f}")
                print(f"  理由: {result.reasoning[:100]}...")
            else:
                print(f"  判断: ✓ 无 bug")
                print(f"  置信度: {result.confidence:.2f}")
                print(f"  理由: {result.reasoning[:100]}...")
            
            results.append({
                'name': name,
                'has_bug': result.has_bug,
                'confidence': result.confidence
            })
            
        except Exception as e:
            print(f"  ✗ 错误: {e}")
            results.append({
                'name': name,
                'has_bug': None,
                'confidence': 0.0
            })
        
        print()
    
    # 统计
    print("=" * 80)
    print("统计结果")
    print("=" * 80)
    
    detected_bugs = sum(1 for r in results if r['has_bug'] == True)
    no_bugs = sum(1 for r in results if r['has_bug'] == False)
    
    print(f"检测到有 bug: {detected_bugs} 个")
    print(f"检测到无 bug: {no_bugs} 个")
    print()
    print("预期结果：前 4 个应该有 bug，第 5 个（安全代码）应该无 bug")
    print()
    
    # 对比预期
    expected = [True, True, True, True, False]
    actual = [r['has_bug'] for r in results]
    
    correct = sum(1 for e, a in zip(expected, actual) if e == a)
    accuracy = correct / len(expected) * 100
    
    print(f"准确率: {correct}/{len(expected)} = {accuracy:.1f}%")
    print("=" * 80)
    
else:
    print("GEPA 原生模型未加载，跳过测试")

NameError: name 'gepa_detector' is not defined

In [7]:
# 加载 GEPA 原生模型
import sys
import os

# 添加路径
sys.path.insert(0, os.path.abspath('.'))

try:
    from bug_detector_gepa_native import BugPresenceDetector
    
    print("=" * 80)
    print("加载 GEPA 原生优化模型")
    print("=" * 80)
    
    gepa_native_path = '../models/gepa_native_presence_detector.json'
    
    if os.path.exists(gepa_native_path):
        gepa_detector = BugPresenceDetector()
        gepa_detector.detect.load(gepa_native_path)
        print(f"✓ 成功加载 GEPA 原生模型: {gepa_native_path}")
        print(f"✓ 模型特点：")
        print(f"  - 任务：判断代码是否包含 bug（二分类）")
        print(f"  - 准确率：80% (基础模型 66.7%)")
        print(f"  - 优化方法：GEPA heavy 模式，69 次迭代")
        print(f"  - 训练时间：约 30 分钟")
        print()
        print("✓ GEPA 原生检测器已就绪")
    else:
        print(f"✗ 未找到 GEPA 原生模型: {gepa_native_path}")
        print(f"  请先运行训练：python python_src/train_gepa_native.py")
        gepa_detector = None
        
    print("=" * 80)
    
except ImportError as e:
    print(f"✗ 无法导入 GEPA 原生模块: {e}")
    print("  请确保 bug_detector_gepa_native.py 在正确的路径")
    gepa_detector = None

加载 GEPA 原生优化模型


KeyError: 'predict'

# C语言代码Bug检测演示

本notebook演示如何使用DSPy实现的代码bug检测功能。

## 1. 环境设置

首先，加载DSPy模型配置和bug检测模块。

### 关于模型优化

本系统支持使用DSPy的BootstrapFewShot优化器来提升检测准确性。优化后的模型：
- ✓ 更准确地识别真正的bug
- ✓ 减少假阳性（误报）
- ✓ 特别是对于复杂的安全检查代码（如示例5）

**如何训练优化模型：**

在终端中运行：
```bash
cd python_src
export GOOGLE_API_KEY='your-api-key'
python train_optimizer.py
```

训练完成后，本notebook会自动使用优化模型。

In [1]:
# 导入必要的库
import dspy
import os
from code_bug_detector import CodeBugDetector, BUG_TYPE_MAPPING

In [ ]:
# 配置DSPy模型 (与DPSy_tes.ipynb中的配置相同)
lm = dspy.LM(
    model='gemini/gemini-3-flash-preview',
    api_key='',
    max_tokens=4000
)

dspy.configure(lm=lm)
print("✓ DSPy模型配置完成")
dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)

✓ DSPy模型配置完成


In [3]:
# 创建bug检测器实例
# 如果存在优化模型，将自动加载；否则使用基础模型
import os

# 检查是否存在优化模型（JSON格式更安全）
model_path = '../models/optimized_bug_detector.json'
gepa_native_path = '../models/gepa_native_presence_detector.json'

if os.path.exists(gepa_native_path):
    print(f"✓ 发现 GEPA 原生优化模型: {gepa_native_path}")
    print("  这是最新的模型，80%准确率，专注于判断有无bug")
    print("  (注意：该模型只判断有无bug，不分类bug类型)")
    use_gepa_native = True
elif os.path.exists(model_path):
    print(f"正在加载优化模型: {model_path}")
    detector = CodeBugDetector(optimized_model_path=model_path)
    use_gepa_native = False
else:
    print("未找到优化模型，使用基础模型")
    print(f"提示: 运行 'python python_src/train_optimizer.py' 来训练优化模型")
    detector = CodeBugDetector()
    use_gepa_native = False

if not use_gepa_native:
    print("✓ Bug检测器已初始化")
    if detector.is_optimized:
        print("✓ 使用优化模型 (性能更好，假阳性更少)")
    else:
        print("⚠ 使用基础模型 (可能存在假阳性)")
else:
    print("\n下面的示例将使用传统模型（带类别分类）")
    print("如果想测试 GEPA 原生模型，请跳转到最后的对比部分")
    detector = CodeBugDetector()
    print("✓ Bug检测器已初始化 (基础模型)")

✓ 发现 GEPA 原生优化模型: ../models/gepa_native_presence_detector.json
  这是最新的模型，80%准确率，专注于判断有无bug
  (注意：该模型只判断有无bug，不分类bug类型)

下面的示例将使用传统模型（带类别分类）
如果想测试 GEPA 原生模型，请跳转到最后的对比部分
✓ Bug检测器已初始化 (基础模型)


## 2. 示例1：内存泄漏检测

In [4]:
code_example_1 = """
void process_data() {
    int* data = (int*)malloc(100 * sizeof(int));
    if (data == NULL) {
        return;
    }
    
    // 处理数据
    for (int i = 0; i < 100; i++) {
        data[i] = i * 2;
    }
    

}
"""

print("检测代码片段 1：")
print(code_example_1)
print("\n开始检测...")

result1 = detector(code_example_1)
print(detector.format_output(result1))

2026/01/17 22:23:26 WARNING dspy.primitives.module: Calling module.forward(...) on CodeBugDetector directly is discouraged. Please use module(...) instead.


检测代码片段 1：

void process_data() {
    int* data = (int*)malloc(100 * sizeof(int));
    if (data == NULL) {
        return;
    }

    // 处理数据
    for (int i = 0; i < 100; i++) {
        data[i] = i * 2;
    }


}


开始检测...
代码Bug检测结果

✗ 检测到以下bug：

Bug #1:
  位置: 第 13 行
  代码: }
  类型: Memory Error: Uncontrolled Resource Consumption - 资源消耗失控
  说明: 函数在退出前未调用 free(data) 释放由 malloc 分配的内存，导致内存泄漏（Resource Consumption）。



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...vx3BuzO2Hlx+qh6TI64I']}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...x3BuzO2Hlx+qh6TI64I']})), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


## 3. 示例2：缓冲区溢出

In [5]:
code_example_2 = """
void copy_string(char* input) {
    char buffer[10];
    strcpy(buffer, input);  
    printf("Copied: %s\n", buffer);
}
"""

print("检测代码片段 2：")
print(code_example_2)
print("\n开始检测...")

result2 = detector(code_example_2)
print(detector.format_output(result2))

2026/01/17 22:23:32 WARNING dspy.primitives.module: Calling module.forward(...) on CodeBugDetector directly is discouraged. Please use module(...) instead.


检测代码片段 2：

void copy_string(char* input) {
    char buffer[10];
    strcpy(buffer, input);  
    printf("Copied: %s
", buffer);
}


开始检测...
代码Bug检测结果

✗ 检测到以下bug：

Bug #1:
  位置: 第 3 行
  代码: strcpy(buffer, input);
  类型: Memory Error: Memory Overflow - 内存溢出
  说明: 使用 strcpy 函数将长度不受控的 input 复制到固定大小（10字节）的 buffer 中，若输入字符串长度超过 buffer 容量，将导致缓冲区溢出（内存溢出）。



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...Htnbeuhw/C6dVNjH75g9']}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...tnbeuhw/C6dVNjH75g9']})), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


## 4. 示例3：空指针解引用

In [6]:
code_example_3 = """
int get_value(int* ptr) {
    return *ptr;
}

int main() {
    int* p = NULL;
    int value = get_value(p);
    printf("Value: %d\n", value);
    return 0;
}
"""

print("检测代码片段 3：")
print(code_example_3)
print("\n开始检测...")

result3 = detector(code_example_3)
print(detector.format_output(result3))

2026/01/17 22:23:37 WARNING dspy.primitives.module: Calling module.forward(...) on CodeBugDetector directly is discouraged. Please use module(...) instead.


检测代码片段 3：

int get_value(int* ptr) {
    return *ptr;
}

int main() {
    int* p = NULL;
    int value = get_value(p);
    printf("Value: %d
", value);
    return 0;
}


开始检测...
代码Bug检测结果

✗ 检测到以下bug：

Bug #1:
  位置: 第 2 行
  代码: return *ptr;
  类型: Memory Error: Null Pointer Dereference - 空指针解引用
  说明: 函数 get_value 接收到一个空指针 ptr 并尝试对其进行解引用，这将导致空指针解引用错误。

Bug #2:
  位置: 第 8 行
  代码: int value = get_value(p);
  类型: Logic Organization: Wrong Function Call Sequence - 函数调用顺序错误
  说明: 在 main 函数中将初始化为 NULL 的指针 p 传递给需要有效指针的函数，触发了后续的解引用错误。



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...urCdiY59p6Wt+jEJ5Bj8']}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...rCdiY59p6Wt+jEJ5Bj8']})), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


## 5. 示例4：数组越界

In [8]:
code_example_4 = """
void fill_array() {
    int arr[10];
    for (int i = 0; i <= 10; i++) {  
        arr[i] = i;
    }
}
"""

print("检测代码片段 4：")
print(code_example_4)
print("\n开始检测...")

result4 = detector(code_example_4)
print(detector.format_output(result4))

2026/01/17 22:24:06 WARNING dspy.primitives.module: Calling module.forward(...) on CodeBugDetector directly is discouraged. Please use module(...) instead.


检测代码片段 4：

void fill_array() {
    int arr[10];
    for (int i = 0; i <= 10; i++) {  
        arr[i] = i;
    }
}


开始检测...
代码Bug检测结果

✗ 检测到以下bug：

Bug #1:
  位置: 第 3 行
  代码: for (int i = 0; i <= 10; i++) {
  类型: Memory Error: Memory Overflow - 内存溢出
  说明: The loop condition 'i <= 10' allows the index to reach 10, but the array 'arr' only has 10 elements (indices 0-9). This results in a buffer overflow when i is 10.



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...XAkvaJY2SvDtufl3Z1cN']}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...AkvaJY2SvDtufl3Z1cN']})), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


## 6. 示例5：正确的代码（无bug）

In [9]:
code_example_5 = """
unsigned int hash_function(unsigned int key) {
    return key * 2654435761u;
}
"""

print("检测代码片段 5：")
print(code_example_5)
print("\n开始检测...")

result5 = detector(code_example_5)
print(detector.format_output(result5))

2026/01/18 13:34:23 WARNING dspy.primitives.module: Calling module.forward(...) on CodeBugDetector directly is discouraged. Please use module(...) instead.


检测代码片段 5：

unsigned int hash_function(unsigned int key) {
    return key * 2654435761u;
}


开始检测...
代码Bug检测结果

✓ 未检测到bug，代码看起来正常。



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...aPlF9wyaVTyi7cnP3ISx']}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...PlF9wyaVTyi7cnP3ISx']})), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


## 7. 自定义代码检测

你可以在下面的单元格中输入自己的C代码进行检测。

In [10]:
# 在这里输入你要检测的C代码
custom_code = """
// 在这里输入你的C代码
"""

if custom_code.strip() and "在这里输入" not in custom_code:
    print("检测自定义代码：")
    print(custom_code)
    print("\n开始检测...")
    
    custom_result = detector(custom_code)
    print(detector.format_output(custom_result))
else:
    print("请先输入要检测的代码")

请先输入要检测的代码


## 8. 查看支持的Bug类型

查看系统支持的所有bug类型及其描述。

In [ ]:
print("支持的Bug类型列表：")
print("=" * 80)
for bug_id, description in BUG_TYPE_MAPPING.items():
    print(f"{bug_id}: {description}")
print("=" * 80)

## 9. 批量检测

如果你有多个代码片段需要检测，可以使用批量检测功能。

In [ ]:
# 示例：批量检测多个代码片段
code_samples = [
    code_example_1,
    code_example_2,
    code_example_3,
    code_example_4,
    code_example_5
]

print("开始批量检测...\n")

for i, code in enumerate(code_samples, 1):
    print(f"\n{'='*80}")
    print(f"代码片段 #{i}")
    print(f"{'='*80}")
    
    result = detector(code)
    print(detector.format_output(result))
    
print("\n批量检测完成！")

## 10. 检测结果统计

统计检测到的bug类型分布。

In [19]:
from collections import Counter

def analyze_detection_results(results):
    """
    分析多个检测结果，统计bug类型分布
    """
    bug_types = []
    total_bugs = 0
    
    for result in results:
        if result['has_bug']:
            bugs = result['bug_details'].get('bugs', [])
            for bug in bugs:
                bug_type_id = bug.get('bug_type_id', 'BT020')
                bug_types.append(bug_type_id)
                total_bugs += 1
    
    # 统计每种bug类型的数量
    bug_counter = Counter(bug_types)
    
    print("Bug类型分布统计：")
    print("=" * 80)
    print(f"总计检测到 {total_bugs} 个bug")
    print("\n各类型bug数量：")
    for bug_id, count in bug_counter.most_common():
        print(f"  {bug_id} ({BUG_TYPE_MAPPING.get(bug_id, 'Unknown')}): {count}")
    print("=" * 80)

# 示例使用（取消注释以运行）
results = [result1, result2, result3, result4, result5]
analyze_detection_results(results)

Bug类型分布统计：
总计检测到 4 个bug

各类型bug数量：
  C.3 (Memory Error: Memory Overflow - 内存溢出): 2
  C.2 (Memory Error: Uncontrolled Resource Consumption - 资源消耗失控): 1
  C.1 (Memory Error: Null Pointer Dereference - 空指针解引用): 1


In [22]:
# 显示对比结果表格
df = pd.DataFrame(comparison_results).T
print("\n" + "=" * 80)
print("三种模型对比结果汇总")
print("=" * 80)
print()
display(df)

# 统计假阳性率（测试4应该无bug）
print("\n" + "=" * 80)
print("关键指标对比")
print("=" * 80)

# 测试4是安全代码，应该识别为无bug
safe_code_test = "测试4: 安全代码（防御性编程）"
print(f"\n{safe_code_test}:")
print("  (正确答案：无bug - 这是防御性编程)")
print()

for model_name in ['基础模型', 'BootstrapFewShot', 'GEPA']:
    if model_name in df.columns:
        result = df.loc[safe_code_test, model_name]
        if "无bug" in str(result):
            print(f"  {model_name:20s}: ✓ 正确识别（无假阳性）")
        elif "未加载" in str(result):
            print(f"  {model_name:20s}: - 未加载")
        else:
            print(f"  {model_name:20s}: ✗ 误报为有bug（假阳性）")

print("\n" + "=" * 80)
print("总结")
print("=" * 80)
print("""
预期结果：
- 基础模型：可能有假阳性，将安全检查代码误判为bug
- BootstrapFewShot：性能提升，假阳性减少
- GEPA：使用Claude Opus评估，预期假阳性最少，准确率最高

GEPA的优势：
1. 使用高阶模型（Claude Opus）来评估和指导优化
2. 能更好地理解代码语义和意图
3. 对防御性编程、安全检查等复杂模式识别更准确
""")


三种模型对比结果汇总



,基础模型,BootstrapFewShot,GEPA
测试1: 空指针解引用,有bug (1个): C.1,有bug (1个): C.1,有bug (1个): C.1
测试2: 内存泄漏,"有bug (2个): C.1, C.2",有bug (1个): C.2,有bug (1个): C.2
测试3: 缓冲区溢出,有bug (1个): C.3,有bug (1个): C.3,有bug (1个): C.3
测试4: 安全代码（防御性编程）,有bug (1个): D.1,有bug (1个): B,有bug (1个): D.1
测试5: 数组越界,有bug (1个): C.3,"有bug (2个): C.3, C.3",有bug (1个): C.3



关键指标对比

测试4: 安全代码（防御性编程）:
  (正确答案：无bug - 这是防御性编程)

  基础模型                : ✗ 误报为有bug（假阳性）
  BootstrapFewShot    : ✗ 误报为有bug（假阳性）
  GEPA                : ✗ 误报为有bug（假阳性）

总结

预期结果：
- 基础模型：可能有假阳性，将安全检查代码误判为bug
- BootstrapFewShot：性能提升，假阳性减少
- GEPA：使用Claude Opus评估，预期假阳性最少，准确率最高

GEPA的优势：
1. 使用高阶模型（Claude Opus）来评估和指导优化
2. 能更好地理解代码语义和意图
3. 对防御性编程、安全检查等复杂模式识别更准确



In [5]:
# 运行对比测试
import pandas as pd
from collections import defaultdict

print("=" * 80)
print("开始对比测试...")
print("=" * 80)
print()

# 存储结果
comparison_results = defaultdict(dict)

models = {
    '基础模型': base_detector,
    'BootstrapFewShot': bootstrap_detector,
    'GEPA': gepa_detector
}

for test_name, code in test_cases.items():
    print(f"\n{test_name}")
    print("-" * 80)
    
    for model_name, model in models.items():
        if model is None:
            comparison_results[test_name][model_name] = "未加载"
            print(f"  {model_name}: 未加载")
            continue
        
        try:
            result = model(code)
            has_bug = result['has_bug']
            num_bugs = len(result['bug_details'].get('bugs', []))
            
            if has_bug:
                bug_types = [b.get('bug_type_id', 'Unknown') 
                           for b in result['bug_details'].get('bugs', [])]
                status = f"有bug ({num_bugs}个): {', '.join(bug_types)}"
            else:
                status = "无bug ✓"
            
            comparison_results[test_name][model_name] = status
            print(f"  {model_name:20s}: {status}")
            
        except Exception as e:
            comparison_results[test_name][model_name] = f"错误: {str(e)[:30]}"
            print(f"  {model_name:20s}: 错误 - {str(e)[:30]}")

print("\n" + "=" * 80)
print("对比测试完成")
print("=" * 80)

2026/01/18 17:04:06 WARNING dspy.primitives.module: Calling module.forward(...) on CodeBugDetector directly is discouraged. Please use module(...) instead.


开始对比测试...


测试1: 空指针解引用
--------------------------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...Aq98+LZFf81jDGOJKCzL']}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...q98+LZFf81jDGOJKCzL']})), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
2026/01/18 17:04:09 WARNING dspy.primitives.module: Calling module.forward(...) on CodeBugDetector directly is discouraged. Please use module(...) instead.


  基础模型                : 有bug (1个): C.1


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...Lb+RCdH2RFf5at2LJgq7']}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...b+RCdH2RFf5at2LJgq7']})), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
2026/01/18 17:04:11 WARNING dspy.primitives.module: Calling module.forward(...) on CodeBugDetector directly is discouraged. Please use module(...) instead.


  BootstrapFewShot    : 有bug (1个): C.1


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...t8TqfkSalKJGa4Hpg3W+']}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...8TqfkSalKJGa4Hpg3W+']})), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
2026/01/18 17:04:13 WARNING dspy.primitives.module: Calling module.forward(...) on CodeBugDetector directly is discouraged. Please use module(...) instead.


  GEPA                : 错误 - 'float' object has no attribut

测试2: 内存泄漏
--------------------------------------------------------------------------------


KeyboardInterrupt: 

In [5]:
# 定义对比测试用的代码片段
test_cases = {
    "测试1: 空指针解引用": """
int* p = NULL;
*p = 10;
""",
    "测试2: 内存泄漏": """
void process() {
    int* data = malloc(100 * sizeof(int));
    // 忘记释放
}
""",
    "测试3: 缓冲区溢出": """
void copy(char* input) {
    char buf[10];
    strcpy(buf, input);
}
""",
    "测试4: 安全代码（防御性编程）": """
int safe_add(int a, int b, int* result) {
    if (result == NULL) return -1;
    if (a > 0 && b > INT_MAX - a) return -1;
    *result = a + b;
    return 0;
}
""",
    "测试5: 数组越界": """
void fill() {
    int arr[10];
    for (int i = 0; i <= 10; i++) {
        arr[i] = i;
    }
}
"""
}

print("准备对比测试用例：")
for name in test_cases.keys():
    print(f"  - {name}")

准备对比测试用例：
  - 测试1: 空指针解引用
  - 测试2: 内存泄漏
  - 测试3: 缓冲区溢出
  - 测试4: 安全代码（防御性编程）
  - 测试5: 数组越界


In [6]:
# 加载三种模型进行对比
print("=" * 80)
print("加载三种不同的模型")
print("=" * 80)

# 模型1: 基础模型（未优化）
base_detector = CodeBugDetector()
print("✓ 模型1: 基础模型（未优化）")

# 模型2: BootstrapFewShot优化模型
bootstrap_model_path = '../models/optimized_bug_detector.json'
if os.path.exists(bootstrap_model_path):
    bootstrap_detector = CodeBugDetector(optimized_model_path=bootstrap_model_path)
    print(f"✓ 模型2: BootstrapFewShot优化模型")
else:
    print(f"✗ 模型2: BootstrapFewShot模型未找到，请先运行训练")
    print(f"   运行: python python_src/train_optimizer.py")
    bootstrap_detector = None

# 模型3: GEPA优化模型
gepa_model_path = '../models/gepa_native_presence_detector.json'
if os.path.exists(gepa_model_path):
    gepa_detector = CodeBugDetector(optimized_model_path=gepa_model_path)
    print(f"✓ 模型3: GEPA优化模型（Claude Opus评估）")
else:
    print(f"⚠ 模型3: GEPA模型未找到，请先运行GEPA训练")
    print(f"   运行: python python_src/train_gepa_optimizer.py")
    gepa_detector = None

print("=" * 80)

加载三种不同的模型
✓ 模型1: 基础模型（未优化）
✓ 已加载优化模型: ../models/optimized_bug_detector.json
✓ 模型2: BootstrapFewShot优化模型
✓ 已加载优化模型: ../models/gepa_native_presence_detector.json
✓ 模型3: GEPA优化模型（Claude Opus评估）


## 11. 模型对比测试

对比三种优化方法的效果：
- **基础模型**：未经优化的原始模型
- **BootstrapFewShot优化模型**：使用Few-Shot Bootstrap优化
- **GEPA优化模型**：使用Claude Opus 4.5作为评估器进行优化

GEPA (Generative Evaluator for Prompt Architecture) 是DSPy最新的优化器，使用高阶模型来评价和优化低阶模型的性能。

## 12. GEPA 原生优化模型测试 🎉

**全新的 GEPA 原生适配架构！**

这是经过完全重构的 Bug 检测系统，专门为 GEPA 优化器设计：

**特点**：
- ✅ **80% 准确率** - Bug 存在判断（基础模型 66.7% → 优化后 80%）
- ✅ **历史性突破** - 第一次让 GEPA 在 Bug 检测任务上真正工作
- ✅ **架构创新** - 消除 JSON 解析依赖，直接使用 Python 对象
- ⚠️ **功能范围** - 只判断有无 bug，不分类 bug 类型

**为什么只判断有无 bug？**
1. Bug 类别分类对于纯 prompt 优化（GEPA 的方法）太困难
2. 即使简化为 3 大类，基础模型也只有 3.3% 准确率
3. 类别分类需要 few-shot 示例，而 GEPA 不提供示例

**下一步**：如果需要类别分类，建议使用 MIPROv2（指令 + Few-shot 混合优化）